In [4]:
import pandas
import os
from zipfile import BadZipFile

In [11]:
# List of all files
def get_files(mode:str):
    return os.listdir(f"../{mode}")

In [6]:
def extract_from_name(filename:str)->dict:
    bank, year, month, mode = filename.split("_", 3)
    bank_code, bank_name = bank.split("-", 1)
    bank_name = bank_name.replace("+", " ")

    # Convert month into quartil
    if month == 3:
        month = "Q1"
    elif month == 6:
        month = "Q2"
    elif month == 9:
        month = "Q3"
    elif month == 12:
        month = "Q4"

    # Create unique ID
    id = f"{bank_code}/{month}/{year}"
    
    return {
        "id": id,
        "bank code": bank_code,
        "bank_name": bank_name,
        "year": int(year),
        "month": month,
    }    

In [5]:
# ROA, ROE, NIM, 
def get_roa(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains("Return on Asset", na=False)]['Unnamed: 6'].dropna().values[0]

def get_roe(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains("Return on Equity", na=False)]['Unnamed: 6'].dropna().values[0]

def get_nim(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains("Net Interest Margin", na=False)]['Unnamed: 6'].dropna().values[0]

# Pendapatan Bunga, Beban Bunga
def get_pendapatan_bunga(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains("Pendapatan Bunga", na=False)]['Unnamed: 4'].values[0]

def get_beban_bunga(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains(r"Beban Bunga", na=False)]['Unnamed: 4'].dropna().values[0]

# Total Aset
def get_aset(df:pandas.DataFrame):
    return df[df["Unnamed: 0"].str.contains("TOTAL ASET", na=False)].iloc[0, -4]


In [17]:
# Extract single file
def single_extract(mode:str, filename:str):
    try:
        base_data = extract_from_name(filename)
        filepath = f"{mode}/{filename}"
        df = pandas.read_excel(f"../{filepath}")

        if mode == "neraca":
            total_aset = get_aset(df)
            base_data['total_aset'] = total_aset
        elif mode == "laba_rugi":
            pendapatan_bunga = get_pendapatan_bunga(df)
            beban_bunga = get_beban_bunga(df)
            base_data["pendapatan_bunga"] = pendapatan_bunga
            base_data["beban_bunga"] = beban_bunga
        elif mode == "rasio":
            roa = get_roa(df)
            roe = get_roe(df)
            nim = get_nim(df)
            base_data["roa"] = roa
            base_data["roe"] = roe
            base_data["nim"] = nim
    except BadZipFile:
        print(f"Corrupted file: {filename}")
        return filepath
    except Exception as e:
        print(Exception(e))
        return filepath
    finally:
        return base_data

# Extract by mode
def extract(mode:str):
    collect_data = []
    error_file = []
    files = get_files(mode)
    max = len(files)
    for i,file in enumerate(files):
        print(f"Processing {i+1} of {max}")
        base_data = single_extract(mode, file)
        if isinstance(base_data, dict):
            collect_data.append(base_data)
        else:
            error_file.append(base_data)
    
    df = pandas.DataFrame(collect_data)
    df.to_excel(f"result_{mode}.xlsx", index=False)

    with open(f"error_{mode}.txt", "w") as file:
        file.writelines(error_file)

    print("Success")

In [16]:
extract("neraca")

Processing 1 of 5488
Processing 2 of 5488
Processing 3 of 5488
Processing 4 of 5488
Processing 5 of 5488
Processing 6 of 5488
Processing 7 of 5488
Processing 8 of 5488
Processing 9 of 5488
Processing 10 of 5488
Processing 11 of 5488
Processing 12 of 5488
Processing 13 of 5488
Processing 14 of 5488
Processing 15 of 5488
Processing 16 of 5488
Processing 17 of 5488
Processing 18 of 5488
Processing 19 of 5488
Processing 20 of 5488
Processing 21 of 5488
Processing 22 of 5488
Processing 23 of 5488
Processing 24 of 5488
Processing 25 of 5488
Processing 26 of 5488
Processing 27 of 5488
Processing 28 of 5488
Processing 29 of 5488
Processing 30 of 5488
Processing 31 of 5488
Processing 32 of 5488
Processing 33 of 5488
Processing 34 of 5488
Processing 35 of 5488
Processing 36 of 5488
Processing 37 of 5488
Processing 38 of 5488
Processing 39 of 5488
Processing 40 of 5488
Processing 41 of 5488
Processing 42 of 5488
Processing 43 of 5488
Processing 44 of 5488
Processing 45 of 5488
Processing 46 of 54

In [18]:
extract("laba_rugi")

Processing 1 of 5493
Processing 2 of 5493
Processing 3 of 5493
index 0 is out of bounds for axis 0 with size 0
Processing 4 of 5493
Processing 5 of 5493
Processing 6 of 5493
Processing 7 of 5493
Processing 8 of 5493
Processing 9 of 5493
Processing 10 of 5493
Processing 11 of 5493
Processing 12 of 5493
Processing 13 of 5493
Processing 14 of 5493
Processing 15 of 5493
Processing 16 of 5493
Processing 17 of 5493
Processing 18 of 5493
Processing 19 of 5493
Processing 20 of 5493
Processing 21 of 5493
Processing 22 of 5493
Processing 23 of 5493
Processing 24 of 5493
Processing 25 of 5493
Processing 26 of 5493
Processing 27 of 5493
Processing 28 of 5493
Processing 29 of 5493
Processing 30 of 5493
Processing 31 of 5493
Processing 32 of 5493
Processing 33 of 5493
Processing 34 of 5493
Processing 35 of 5493
Processing 36 of 5493
Processing 37 of 5493
Processing 38 of 5493
Processing 39 of 5493
Processing 40 of 5493
Processing 41 of 5493
Processing 42 of 5493
Processing 43 of 5493
Processing 44 o

In [19]:
extract("rasio")

Processing 1 of 5491
Processing 2 of 5491
Processing 3 of 5491
Processing 4 of 5491
Processing 5 of 5491
Processing 6 of 5491
Processing 7 of 5491
Processing 8 of 5491
Processing 9 of 5491
Processing 10 of 5491
Processing 11 of 5491
Processing 12 of 5491
Processing 13 of 5491
Processing 14 of 5491
Processing 15 of 5491
Processing 16 of 5491
Processing 17 of 5491
Processing 18 of 5491
Processing 19 of 5491
Processing 20 of 5491
Processing 21 of 5491
Processing 22 of 5491
index 0 is out of bounds for axis 0 with size 0
Processing 23 of 5491
Processing 24 of 5491
Processing 25 of 5491
Processing 26 of 5491
Processing 27 of 5491
Processing 28 of 5491
Processing 29 of 5491
Processing 30 of 5491
Processing 31 of 5491
Processing 32 of 5491
Processing 33 of 5491
Processing 34 of 5491
Processing 35 of 5491
Processing 36 of 5491
Processing 37 of 5491
Processing 38 of 5491
index 0 is out of bounds for axis 0 with size 0
Processing 39 of 5491
Processing 40 of 5491
index 0 is out of bounds for axis

In [20]:
# After all the output file is completed merge all
neraca_df = pandas.read_excel("result_neraca.xlsx")
laba_df = pandas.read_excel("result_laba_rugi.xlsx")
rasio_df = pandas.read_excel("result_rasio.xlsx")



In [23]:
merge_df = pandas.merge(neraca_df, laba_df, on=["id", "bank code", "bank_name", "year", "month"])
merge_df

,id,bank code,bank_name,year,month,total_aset,pendapatan_bunga,beban_bunga
0,490/9/2016,490,PT BANK NEO COMMERCE TBK,2016,9,3767592.0,358863.0,197375.0
1,126/6/2011,126,PT BPD SULAWESI SELATAN DAN SULAWESI BARAT,2011,6,7807449.0,510536.0,202156.0
2,501/3/2022,501,PT BANK DIGITAL BCA,2022,3,6904132.0,66881.0,17863.0
3,047/9/2019,47,PT BANK RESONA PERDANIA,2019,9,17895164.0,787331.0,454161.0
4,046/9/2019,46,PT BANK DBS INDONESIA,2019,9,93948133.0,5371238.0,2358697.0
...,...,...,...,...,...,...,...,...
5474,113/12/2012,113,PT BPD JAWA TENGAH,2012,12,26483003.0,2990374.0,0.0
5475,114/9/2011,114,PT BPD JAWA TIMUR Tbk,2011,9,21792640.0,1980833.0,565507.0
5476,145/12/2017,145,"PT BANK NUSANTARA PARAHYANGAN,Tbk",2017,12,7581032.0,789862.0,355906.0
5477,146/3/2019,146,"PT BANK OF INDIA INDONESIA, Tbk",2019,3,3832001.0,70562.0,34725.0


In [ ]:
merge_df = pandas.merge(merge_df, rasio_df, on=["id", "bank code", "bank_name", "year", "month"])
merge_df

,id,bank code,bank_name,year,month,total_aset,pendapatan_bunga,beban_bunga,roa,roe,nim
0,490/9/2016,490,PT BANK NEO COMMERCE TBK,2016,9,3767592.0,358863.0,197375.0,2.66,16.49,6.78
1,126/6/2011,126,PT BPD SULAWESI SELATAN DAN SULAWESI BARAT,2011,6,7807449.0,510536.0,202156.0,4.29,14.14,8.49
2,501/3/2022,501,PT BANK DIGITAL BCA,2022,3,6904132.0,66881.0,17863.0,-1.38,-2.26,3.16
3,047/9/2019,47,PT BANK RESONA PERDANIA,2019,9,17895164.0,787331.0,454161.0,0.76,5.56,2.75
4,046/9/2019,46,PT BANK DBS INDONESIA,2019,9,93948133.0,5371238.0,2358697.0,0.29,2.58,5.06
...,...,...,...,...,...,...,...,...,...,...,...
5460,113/12/2012,113,PT BPD JAWA TENGAH,2012,12,26483003.0,2990374.0,0.0,2.73,30.69,8.22
5461,114/9/2011,114,PT BPD JAWA TIMUR Tbk,2011,9,21792640.0,1980833.0,565507.0,5.29,36.25,7.94
5462,145/12/2017,145,"PT BANK NUSANTARA PARAHYANGAN,Tbk",2017,12,7581032.0,789862.0,355906.0,-0.90,-5.27,6.12
5463,146/3/2019,146,"PT BANK OF INDIA INDONESIA, Tbk",2019,3,3832001.0,70562.0,34725.0,1.37,3.79,4.23


In [26]:
month_to_quarter = {3: 'Q1', 6: 'Q2', 9: 'Q3', 12: 'Q4'}
merge_df['Quarter'] = merge_df['month'].map(month_to_quarter)
merge_df



,id,bank code,bank_name,year,month,total_aset,pendapatan_bunga,beban_bunga,roa,roe,nim,Quarter
0,490/9/2016,490,PT BANK NEO COMMERCE TBK,2016,9,3767592.0,358863.0,197375.0,2.66,16.49,6.78,Q3
1,126/6/2011,126,PT BPD SULAWESI SELATAN DAN SULAWESI BARAT,2011,6,7807449.0,510536.0,202156.0,4.29,14.14,8.49,Q2
2,501/3/2022,501,PT BANK DIGITAL BCA,2022,3,6904132.0,66881.0,17863.0,-1.38,-2.26,3.16,Q1
3,047/9/2019,47,PT BANK RESONA PERDANIA,2019,9,17895164.0,787331.0,454161.0,0.76,5.56,2.75,Q3
4,046/9/2019,46,PT BANK DBS INDONESIA,2019,9,93948133.0,5371238.0,2358697.0,0.29,2.58,5.06,Q3
...,...,...,...,...,...,...,...,...,...,...,...,...
5460,113/12/2012,113,PT BPD JAWA TENGAH,2012,12,26483003.0,2990374.0,0.0,2.73,30.69,8.22,Q4
5461,114/9/2011,114,PT BPD JAWA TIMUR Tbk,2011,9,21792640.0,1980833.0,565507.0,5.29,36.25,7.94,Q3
5462,145/12/2017,145,"PT BANK NUSANTARA PARAHYANGAN,Tbk",2017,12,7581032.0,789862.0,355906.0,-0.90,-5.27,6.12,Q4
5463,146/3/2019,146,"PT BANK OF INDIA INDONESIA, Tbk",2019,3,3832001.0,70562.0,34725.0,1.37,3.79,4.23,Q1


In [27]:
# Sort by bank code
merge_df.sort_values(by='bank code', ascending=True, inplace=True)

In [29]:
grouped = merge_df.groupby('bank code').apply(lambda x: x.sort_values(by=['year', 'month']))
grouped


/tmp/ipykernel_3156841/1609054638.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = merge_df.groupby('bank code').apply(lambda x: x.sort_values(by=['year', 'month']))


id  bank code  \
bank code                                
2         3728   002/6/2010          2   
          2811   002/9/2010          2   
          4632  002/12/2010          2   
          2638   002/3/2011          2   
          1824   002/6/2011          2   
...                     ...        ...   
950       255   950/12/2022        950   
          1781   950/3/2023        950   
          493    950/6/2023        950   
          1469   950/9/2023        950   
          167   950/12/2023        950   

                                              bank_name  year  month  \
bank code                                                              
2         3728  PT BANK RAKYAT INDONESIA (PERSERO), Tbk  2010      6   
          2811  PT BANK RAKYAT INDONESIA (PERSERO), Tbk  2010      9   
          4632  PT BANK RAKYAT INDONESIA (PERSERO), Tbk  2010     12   
          2638  PT BANK RAKYAT INDONESIA (PERSERO), Tbk  2011      3   
          1824  PT BANK RAKYAT INDONESIA (PERSERO), Tbk  2011      6   
...                                                 ...   ...    ...   
950       255                     PT BANK COMMONWEALTH   2022     12   
          1781                    PT BANK COMMONWEALTH   2023      3   
          493                     PT BANK COMMONWEALTH   2023      6   
          1469                    PT BANK COMMONWEALTH   2023      9   
          167                     PT BANK COMMONWEALTH   2023     12   

                 total_aset  pendapatan_bunga  beban_bunga   roa    roe  \
bank code                                                                 
2         3728          NaN        19119416.0    5568605.0  3.51  33.41   
          2811  320835587.0        29107797.0    8352666.0  3.65  34.28   
          4632  398393138.0        43971493.0   11449686.0  4.64  43.83   
          2638  366733961.0        11335314.0    3254049.0  4.41  37.73   
          1824  370303369.0        23070217.0    6424734.0  4.44  39.11   
...                     ...               ...          ...   ...    ...   
950       255    18394555.0         1033817.0     347174.0 -1.97 -11.10   
          1781   17760348.0          246195.0      94563.0 -3.81 -16.45   
          493    17899546.0          492854.0     195087.0 -3.83 -16.74   
          1469   16553651.0          734473.0     298898.0 -3.92 -17.22   
          167    16029165.0          952653.0     387502.0 -4.67 -24.49   

                  nim Quarter  
bank code                      
2         3728   9.40      Q2  
          2811   9.50      Q3  
          4632  10.77      Q4  
          2638   9.67      Q1  
          1824   9.88      Q2  
...               ...     ...  
950       255    4.09      Q4  
          1781   3.96      Q1  
          493    3.91      Q2  
          1469   3.85      Q3  
          167    3.84      Q4  

[5465 rows x 12 columns]

In [30]:
grouped.to_excel("results.xlsx", index=False)